# A/B-тестирование

Получены данные проведенного A/B-тестирования рекомендательной системы интернет-магазина. 

### Цели

- проверить корректность проведения теста
- проанализировать полученные результаты

### Техническое задание

- Название теста: `recommender_system_test`;
- Группы: А (контрольная), B (новая платежная воронка);
- Дата запуска: 2020-12-07
- Дата остановки набора новых пользователей: 2020-12-21;
- Дата остановки: 2021-01-04
- Аудитория: 15% новых пользователей из региона EU;
- Назначение теста: тестирование изменений, связанных с внедрением улучшенной рекомендательной системы;
- Ожидаемый эффект: за 14 дней с момента регистрации в системе пользователи покажут лучшую конверсию в просмотр карточек товаров (событие `product_page`), просмотр  корзины товаров (событие `product_card`) и покупку (`purchase`). На каждом из шагов воронки `product_page → product_card → purchase` улучшение составит не менее 10%;
- Ожидаемое количество участников теста: 6000.

### Описание данных

**`ab_project_marketing_events.csv`** — календарь маркетинговых событий на 2020 год:
- `name` — название маркетингового события
- `regions` — регионы, в которых будет проводиться рекламная кампания 
- `start_dt` — дата начала кампании
- `finish_dt` — дата завершения кампании


**`final_ab_new_users.csv`** — все пользователи, зарегистрировавшиеся в интернет-магазине в период с 7 по 21 декабря 2020 года
- `user_id` — уникальный идентификатор пользователя
- `first_date` — дата регистрации
- `region` — регион, к которому относится пользователь
- `device` — устройство, с которого происходила регистрация


**`final_ab_events.csv`** — все события новых пользователей в период с 7 декабря 2020 по 4 января 2021 года
- `user_id` — уникальный идентификатор пользователя
- `event_dt` — дата и время покупки
- `event_name` — наименование типа события
- `details` — дополнительные данные о событии (Ex. стоимость покупки (USD) для события «покупка» (`purchase`))


**`final_ab_participants.csv`** — таблица участников тестов
- `user_id` — уникальный идентификатор пользователя
- `ab_test` — название теста
- `group` — группа, в которой находился пользователь в рамках теста

### План

1. Исследование данных
  - marketing_events
  - new_users
  - final_events
  - final_participants
    
    
2. Исследовательский анализ данных


4. Выводы

## 1. Исследование данных

In [1]:
# импорт библиотек
import pandas as pd

In [2]:
# загружаем файлы
marketing_events = pd.read_csv('datasets/ab_project_marketing_events.csv')
new_users = pd.read_csv('datasets/final_ab_new_users.csv')
final_events = pd.read_csv('datasets/final_ab_events.csv')
final_participants = pd.read_csv('datasets/final_ab_participants.csv')

##### - marketing_events

In [3]:
marketing_events.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14 entries, 0 to 13
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   name       14 non-null     object
 1   regions    14 non-null     object
 2   start_dt   14 non-null     object
 3   finish_dt  14 non-null     object
dtypes: object(4)
memory usage: 576.0+ bytes


Пропуски и дубликаты отсутствуют. Требуется преобразование типов данных в столбцах _start_dt_ и _finish_dt_ в тип даты/времени

Выполним преобразование

In [4]:
# преобразовываем типы данных
for column_name in ['start_dt', 'finish_dt']:
    marketing_events[column_name] = pd.to_datetime(marketing_events[column_name])

##### - new_users

In [5]:
new_users.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 61733 entries, 0 to 61732
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   user_id     61733 non-null  object
 1   first_date  61733 non-null  object
 2   region      61733 non-null  object
 3   device      61733 non-null  object
dtypes: object(4)
memory usage: 1.9+ MB


Проверим дубликаты в идентификаторах пользователей

In [6]:
new_users.duplicated('user_id').unique()

array([False])

Пропуски отсутствуют. Нет задублировавшихся записей о пользователях. Требуется преобразование типов данных в столбце _first_date_ в тип даты/времени

Выполним преобразование

In [7]:
new_users['first_date'] = pd.to_datetime(new_users['first_date'])

##### - final_events

In [8]:
final_events.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 440317 entries, 0 to 440316
Data columns (total 4 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   user_id     440317 non-null  object 
 1   event_dt    440317 non-null  object 
 2   event_name  440317 non-null  object 
 3   details     62740 non-null   float64
dtypes: float64(1), object(3)
memory usage: 13.4+ MB


Проверим дубликаты в идентификаторах пользователей

In [9]:
final_events['user_id'].duplicated().unique()

array([False,  True])

Проверим строки, в которых появляются пропуске в столбце _details_

In [10]:
final_events[final_events['details'].isna()]['event_name'].unique()

array(['product_cart', 'product_page', 'login'], dtype=object)

Присутствуют дубликаты пользователей - скорее всего действия совершались в разное время. Полные дубликаты строк отсутствуют

Имеются пропуски в столбце _details_. Исходя из анализа, данный столбец предназначен для обозначения суммы покупки

Столбец _event_dt_ требует преобразования типов в тип даты/времени

Выполним преобразование

In [11]:
final_events['event_dt'] = pd.to_datetime(final_events['event_dt'])

Найдем первое время совершения каждого действия для каждого пользователя

In [12]:
user_first_event = final_events.pivot_table(index=['user_id', 'event_name'], values='event_dt',
                                           aggfunc='min').sort_values('event_dt')

In [13]:
user_first_event.head(12)

event_dt
user_id          event_name                      
EB2D1E4A76B38E24 purchase     2020-12-07 00:00:33
                 login        2020-12-07 00:00:33
                 product_page 2020-12-07 00:00:34
821D82C2BB74CBB2 product_page 2020-12-07 00:00:35
                 login        2020-12-07 00:00:35
CB807128858C6274 login        2020-12-07 00:01:19
7F256CA122A4084C login        2020-12-07 00:01:37
                 product_cart 2020-12-07 00:01:38
                 product_page 2020-12-07 00:01:40
E4BF0599D6E802CA login        2020-12-07 00:02:46
                 purchase     2020-12-07 00:02:46
                 product_page 2020-12-07 00:02:47

Некорректно прописывается тайминг действий - у пользователей часто покупка идет перед показом страницы с корзиной

##### - final_participants

In [14]:
final_participants.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18268 entries, 0 to 18267
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   user_id  18268 non-null  object
 1   group    18268 non-null  object
 2   ab_test  18268 non-null  object
dtypes: object(3)
memory usage: 428.3+ KB


In [15]:
final_participants['user_id'].duplicated().unique()

array([False,  True])

Имеются дубли в идентификаторах пользователей - такие пользователи участвовали в двух тестах одновременно

## 2. Исследовательский анализ данных

Необходимо выделить пользователей, которые участвовали только в одном тесте, и разделить их на группы A и B

In [16]:
# считаем количество тестов, в которых приял участие пользователь
check_for_2_tests = final_participants.groupby('user_id').agg({'ab_test':'count'}).reset_index()

# создаем список пользователей, участвовавших в двух тестах
users_in_2_tests = check_for_2_tests[check_for_2_tests['ab_test'] == 2]['user_id']

Раз пользователи участвовали в двух тестах, то необходимо удалить их из выборки, так как невоозможно сказать, что повлияло на увеличение конверсии - один тест или второй

In [17]:
# чистим пользователей из двух тестов и формируем выборку необходимых пользователей
clear_final_participants = final_participants\
    .query('user_id not in @users_in_2_tests and ab_test != "interface_eu_test"')

Разделим выборку на группы теста

In [18]:
# A
groupA = clear_final_participants.query('group == "A"')[['user_id']]

# B
groupB = clear_final_participants.query('group == "B"')[['user_id']]

Проверим, не попадают ли пользователи из одной группы в другую

In [19]:
groupA.merge(groupB, on='user_id')

,user_id


Разделение на группы происходит верно

По техническому заданию, пользователи за 14 дней с момента регистрации показать увеличение конверсии. Тест начался 7.12, а 25.12 была проведена маркетинговая акция. Поэтому, примем гипотезу, что на пользователей после определенной даты может повлиять акция. Отделим таких пользователей. Выберем дату X - 24.12. С учетом условия о 14-ти днях, пользователи должны зарегистрироваться до 11.12 включительно, чтобы не попасть под влияние рождественской акции

Также, это должны быть пользователи из Евросоюза

Выделим таких пользователей

In [20]:
checked_new_users = new_users.query('region == "EU" & first_date <= "2020-12-11"')

Найдем пользователей, принявших участие в тесте и удовлетворяющих нашим условиям

In [21]:
# A
groupA_clear = groupA.query('user_id in @checked_new_users.user_id')

# B
groupB_clear = groupB.query('user_id in @checked_new_users.user_id')

Сравним выборки

In [22]:
print('Размер группы A:', groupA_clear.shape[0])
print('Размер группы B:', groupB_clear.shape[0])
print()
print('Отличие группы B от A: {:.0%}'.format(abs(groupB_clear.shape[0] / groupA_clear.shape[0] - 1)))

Размер группы A: 749
Размер группы B: 591

Отличие группы B от A: 21%


Разница в долях очень большая для правильной интерпретации результатов теста


Считаю продолжение анализа нецелесообразным

## 3. Вывод

Считаю дальнейшее продолжение анализа нецелесообразным:


  1. Неверно выбрано время проведения теста:
    - магазин иностранный, тест проводят на пользователях из Евросоюза. Это значит, что в декабре будет подъем спроса на покупки из-за Рождества 25.12. Следовательно, сказать с уверенностью, что повлияло на увеличение конверсии - новая система рекомендаций или праздники - невозможно
    - 25.12 был проведен маркетинговый ивент, который также направлен на привлечение пользователей и скорее всего повлиял на увеличение конверсии
    
    
  2. Неверное логирование событий:
    - часто покупка происходит раньше, чем просмотр корзины
    
    
  3. Участие одних и тех же пользователей в разных тестах (recommender_system_test и interface_eu_test). Очевидно, на таких пользователей могли повлиять и изменения интерфейса и системы рекомендаций


  3. Отсутствие проведенного A/A теста для проверки правильности деления на группы и распределения траффика
  
  
  4. Некорректное деление трафика между группой A и B:
    - различие между долями составляет 21%
    

Основываясь на вышесказанном, предлагаю признать тест некорректным и непоказательным. Дальнейшая проработка и интерпретация результатов повлечет за собой неправильные выводы, которые могут привести к неправильным решениям